In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import matplotlib.pyplot as plt
import math
import scipy.stats
pitorch = torch.Tensor([math.pi])
import pickle
import pandas as pd
from pathlib import Path
from distmesh import *
from FEM_TPT import *


def dU(x,y):
    a = torch.tensor([-1,-1,-6.5,0.7])
    b = torch.tensor([0,0,11,0.6])
    c = torch.tensor([-10,-10,-6.5,0.7])
    D = torch.tensor([-200,-100,-170,15])
    X = torch.tensor([1,0,-0.5,-1])
    Y = torch.tensor([0,0.5,1.5,1])
    gamma = torch.tensor([9])
    k = torch.tensor([5])
  
    fx1 = D[0]*torch.exp(a[0]*((x-X[0]).pow(2)) + b[0]*(x-X[0])*(y-Y[0]) + c[0]*(y.pow(2)))
    fx2 = D[1]*torch.exp(a[1]*((x-X[1]).pow(2)) + b[1]*(x-X[1])*(y-Y[1]) + c[1]*((y-Y[1]).pow(2)))
    fx3 = D[2]*torch.exp(a[2]*((x-X[2]).pow(2)) + b[2]*(x-X[2])*(y-Y[2]) + c[2]*((y-Y[2]).pow(2)))
    fx4 = D[3]*torch.exp(a[3]*((x-X[3]).pow(2)) + b[3]*(x-X[3])*(y-Y[3]) + c[3]*((y-Y[3]).pow(2)))
    extra = gamma*torch.sin(2*k*pitorch*x)*torch.sin(2*k*pitorch*y)
    
    extrapx = gamma*torch.cos(2*k*pitorch*x)*torch.sin(2*k*pitorch*y)*2*pitorch*k
    extrapy = gamma*torch.cos(2*k*pitorch*y)*torch.sin(2*k*pitorch*x)*2*pitorch*k
    
    dUx = fx1*(2*a[0]*(x-X[0])+b[0]*(y-Y[0])) + fx2*(2*a[1]*(x-X[1])+b[1]*(y-Y[1])) + fx3*(2*a[2]*(x-X[2])+b[2]*(y-Y[2])) + fx4*(2*a[3]*(x-X[3])+b[3]*(y-Y[3]))
    
    dUy = fx1*(2*c[0]*y+b[0]*(x-X[0])) + fx2*(2*c[1]*(y-Y[1])+b[1]*(x-X[1])) + fx3*(2*c[2]*(y-Y[2])+b[2]*(x-X[2])) + fx4*(2*c[3]*(y-Y[3])+b[3]*(x-X[3]))
    
    return dUx, dUy

def funU(x,y):
    a = torch.tensor([-1,-1,-6.5,0.7])
    b = torch.tensor([0,0,11,0.6])
    c = torch.tensor([-10,-10,-6.5,0.7])
    D = torch.tensor([-200,-100,-170,15])
    X = torch.tensor([1,0,-0.5,-1])
    Y = torch.tensor([0,0.5,1.5,1])
    gamma = torch.tensor([9])
    k = torch.tensor([5])
  
    fx1 = D[0]*torch.exp(a[0]*((x-X[0]).pow(2)) + b[0]*(x-X[0])*(y-Y[0]) + c[0]*(y.pow(2)))
    fx2 = D[1]*torch.exp(a[1]*((x-X[1]).pow(2)) + b[1]*(x-X[1])*(y-Y[1]) + c[1]*((y-Y[1]).pow(2)))
    fx3 = D[2]*torch.exp(a[2]*((x-X[2]).pow(2)) + b[2]*(x-X[2])*(y-Y[2]) + c[2]*((y-Y[2]).pow(2)))
    fx4 = D[3]*torch.exp(a[3]*((x-X[3]).pow(2)) + b[3]*(x-X[3])*(y-Y[3]) + c[3]*((y-Y[3]).pow(2)))
    extra = gamma*torch.sin(2*k*pitorch*x)*torch.sin(2*k*pitorch*y)
    
    U = fx1+fx2+fx3+fx4
#     U = U+extra
    
    return U
# Indicator functions Chi_A and Chi_B
def chiAB(X):
    a = torch.tensor([-0.558,1.441])
    b = torch.tensor([0.623,0.028])
    r = torch.tensor(0.1)
    m = nn.Tanh()
    sizex, nothing = X.shape
    chiA = 0.5 - 0.5*m(1000*((((X - a).pow(2)).sum(dim = 1).reshape(sizex,1))-(r + torch.tensor(0.02)).pow(2)))                     
    chiB = 0.5 - 0.5*m(1000*((((X - b).pow(2)).sum(dim = 1).reshape(sizex,1))-(r + torch.tensor(0.02)).pow(2)))       
                             
    return chiA, chiB

def q_theta(X,chiA,chiB,q_tilde):
    Q = (torch.tensor([1]) - chiA)*(q_tilde*(torch.tensor([1]) - chiB)+chiB)
    return Q

def dU_control(beta,q,gradq):
    control_x = torch.tensor(2)/beta*(gradq[:,0]/q)
    control_y = torch.tensor(2)/beta*(gradq[:,1]/q)
    
    return control_x,control_y

In [2]:
class Ruggedmueller2(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        # hidden layer
        self.linear1 = nn.Linear(in_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        # output layer
        self.linear3 = nn.Linear(hidden_size, out_size)
        
    def forward(self, xb):
        # Get intermediate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        tanhf = nn.Tanh()
        out = tanhf(out)
        # Get predictions using output layer
        out = self.linear2(out)
        out = tanhf(out)
        out = self.linear3(out)
        # apply activation function again
        out = torch.sigmoid(out)
        return out

In [3]:
x0 = torch.tensor([[-0.558]])
y0 = torch.tensor([[1.441]])

rsquare = torch.tensor([0.1]).pow(2)
def get_init_pt(circle_r):
    # center of the circle (x, y)
    circle_x = -0.558
    circle_y = 1.441

    # random angle
    alpha = 2 * math.pi * np.random.random()
    # random radius
    r = circle_r * math.sqrt(np.random.random())
    # calculating coordinates
    x = r * math.cos(alpha) + circle_x
    y = r * math.sin(alpha) + circle_y
    
    return torch.tensor([[x]]),torch.tensor([[y]])
def running_traj(Time,M,beta,delt):
    print(Time)
    x0 = torch.tensor([[-0.65]])
    y0 = torch.tensor([[1.441]])
    I = (Time+100)*torch.ones(M)
    for m in range(M):
        
#         x0,y0 = get_init_pt(0.1)
        
        print(m)
        newX = x0
        newY = y0

        w = torch.randn(Time,2)
        w = torch.sqrt(delt)*w
        
        

        flag = 0

        for i in range(Time):

            b = torch.tensor([0.623,0.028])

            newR = torch.cat((newX, newY), 1)

            newR.requires_grad_(True)

            dUx,dUy = dU(newX,newY)
            chia,chib = chiAB(newR)
            q_tilde = model(newR)
            Q = q_theta(newR,chia,chib,q_tilde)
            derivQ = torch.autograd.grad(Q,newR,allow_unused=True, retain_graph=True, \
                                             grad_outputs = torch.ones_like(Q), create_graph=True)

            dU_control_x, dU_control_y = dU_control(beta,Q,derivQ[0])

            newX = newX - (dUx - dU_control_x)*delt + torch.sqrt(2*beta**(-1))*w[i,0]
            newY = newY - (dUy - dU_control_y)*delt + torch.sqrt(2*beta**(-1))*w[i,1]


            # from the time when the trajectory hit B, values in list chi_B will become one
            if flag == 0:
                distB = (newX - b[0]).pow(2) + (newY - b[1]).pow(2)
                if distB <= rsquare:
                    flag = 1
                    I[m] = i 
                    print('we break at: ', i)
                    break
    return I 

In [4]:
beta = torch.tensor(1/10)
delt = torch.tensor(1e-5)
# I = running_traj(20000,250,beta,delt)
# with open('I_250_controlled_T10',"wb") as fp:   #Pickling
#     pickle.dump(I, fp)

In [6]:
model_10 = torch.load('./data/d-net_T10_2hidden40.pt')
model_10.eval()

Ruggedmueller2(
  (linear1): Linear(in_features=2, out_features=40, bias=True)
  (linear2): Linear(in_features=40, out_features=40, bias=True)
  (linear3): Linear(in_features=40, out_features=1, bias=True)
)

In [8]:
def mueller(x):
    a = np.array([-1,-1,-6.5,0.7])
    b = np.array([0,0,11,0.6])
    c = np.array([-10,-10,-6.5,0.7])
    D = np.array([-200,-100,-170,15])
    X = np.array([1,0,-0.5,-1])
    Y = np.array([0,0.5,1.5,1])
    # gamma = torch.tensor([9])
    # k = torch.tensor([5])
  
    fx1 = D[0]*np.exp(a[0]*((x[:,0]-X[0])**(2)) + b[0]*(x[:,0]-X[0])*(x[:,1]-Y[0]) + c[0]*((x[:,1]-Y[0])**(2)))
    fx2 = D[1]*np.exp(a[1]*((x[:,0]-X[1])**(2)) + b[1]*(x[:,0]-X[1])*(x[:,1]-Y[1]) + c[1]*((x[:,1]-Y[1])**(2)))
    fx3 = D[2]*np.exp(a[2]*((x[:,0]-X[2])**(2)) + b[2]*(x[:,0]-X[2])*(x[:,1]-Y[2]) + c[2]*((x[:,1]-Y[2])**(2)))
    fx4 = D[3]*np.exp(a[3]*((x[:,0]-X[3])**(2)) + b[3]*(x[:,0]-X[3])*(x[:,1]-Y[3]) + c[3]*((x[:,1]-Y[3])**(2)))
    # extra = gamma*torch.sin(2*k*pitorch*x)*torch.sin(2*k*pitorch*y)
    
    U = fx1+fx2+fx3+fx4
#     U = U+extra
    return U

In [12]:
data_folder = Path('./data')
pts = np.loadtxt(data_folder/'mueller_pts.csv', delimiter=',', dtype=float)
tri = np.loadtxt(data_folder/'mueller_tri.csv', delimiter=',', dtype=int)
pts_Amesh = np.loadtxt(data_folder/'A_pts.csv', delimiter=',', dtype=float)
tri_Amesh = np.loadtxt(data_folder/'A_tri.csv', delimiter=',', dtype=int)
pts_Bmesh = np.loadtxt(data_folder/'B_pts.csv', delimiter=',', dtype=float)
tri_Bmesh = np.loadtxt(data_folder/'B_tri.csv', delimiter=',', dtype=int)

Z = invariant_pdf(pts,tri,pts_Amesh,tri_Amesh,pts_Bmesh,tri_Bmesh,mueller,beta)

pts_tensor = torch.tensor(pts,dtype=torch.float32)
chia,chib = chiAB(pts_tensor)
q_tilde = model_10(pts_tensor)
q = q_theta(pts_tensor,chia,chib,q_tilde).detach().numpy()
rho_AB_10 = probability_reactive(pts,tri,mueller,beta,q,Z)
rho_AA_10 = probability_last_A_return_A(pts,tri,pts_Amesh,tri_Amesh,mueller,beta,q,Z)
vR_10 = reactive_current_and_transition_rate(pts,tri,mueller,beta,q,Z)

print("""BY NN: rho_AA is {}, rho_AB is {},
transition rate is {}""".format(rho_AA_10.detach().numpy(), rho_AB_10.detach().numpy(),
                                vR_10.detach().numpy()))

BY NN: rho_AA is [0.97438382], rho_AB is [0.000226],
transition rate is [0.00491788]


In [13]:
def mean_confidence_interval(data, confidence=0.95):
    n = len(data)
    m, se = np.mean(data), scipy.stats.sem(data)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [22]:
with open('./data/I_250_controlled_T10','rb') as f:
    I = pickle.load(f)
I = I.detach().numpy()
delt = 1e-5
average_time_steps,left,right = mean_confidence_interval(I)
average_time = average_time_steps*delt
print('E_tau_AB: {}'.format(average_time))

E_tau_AB: 0.0467316015625


In [24]:
rho_AB_10 = rho_AB_10.detach().numpy()
transition_rate_10 = rho_AB_10/average_time
print('estimated transition rate at 10 : {} with interval:[{},{}]'.format(transition_rate_10,
                                                                          rho_AB_10/(right*delt),
                                                                          rho_AB_10/(left*delt)))
print((average_time_steps-left)*delt)
print(transition_rate_10 - rho_AB_10/(right*delt))

estimated transition rate at 10 : [0.00483603] with interval:[[0.00444953],[0.00529607]]
0.004059255833969528
[0.0003865]


# Sampled path graph

In [30]:
def plot_traj(Time,M,beta,delt):
    print(Time)
    a = torch.tensor([-0.558,1.441])
    
    I = (Time+100)*torch.ones(M)
    for m in range(M):
        x0,y0 = get_init_pt(0.3)
#         print(m)
        newX = x0
        newY = y0
        
        newX_orig = x0
        newY_orig = y0

        w = torch.randn(Time,2)
        w = torch.sqrt(delt)*w
        
        flag = 0
        
        X = torch.tensor([])
        Y = torch.tensor([])
        
        X = torch.cat((X,x0), 0)
        Y = torch.cat((Y,y0), 0)
        
        X_orig = torch.tensor([])
        Y_orig = torch.tensor([])
        
        X_orig = torch.cat((X_orig,x0), 0)
        Y_orig = torch.cat((Y_orig,y0), 0)

        for i in range(Time):

            b = torch.tensor([0.623,0.028])

            newR = torch.cat((newX, newY), 1)

            newR.requires_grad_(True)

            dUx,dUy = dU(newX,newY)
            chia,chib = chiAB(newR)
            q_tilde = model_10(newR)
            Q = q_theta(newR,chia,chib,q_tilde)
            derivQ = torch.autograd.grad(Q,newR,allow_unused=True, retain_graph=True, \
                                             grad_outputs = torch.ones_like(Q), create_graph=True)

            dU_control_x, dU_control_y = dU_control(beta,Q,derivQ[0])

            newX = newX - (dUx - dU_control_x)*delt + torch.sqrt(2*beta**(-1))*w[i,0]
            newY = newY - (dUy - dU_control_y)*delt + torch.sqrt(2*beta**(-1))*w[i,1]
            
            X = torch.cat((X,newX), 0)
            Y = torch.cat((Y,newY), 0)
            
            dUx_orig,dUy_orig = dU(newX_orig,newY_orig)
            
            newX_orig = newX_orig - dUx_orig*delt + torch.sqrt(2*beta**(-1))*w[i,0]
            newY_orig = newY_orig - dUy_orig*delt + torch.sqrt(2*beta**(-1))*w[i,1]
            
            X_orig = torch.cat((X_orig,newX_orig), 0)
            Y_orig = torch.cat((Y_orig,newY_orig), 0)
            
        if m == 0:
            plt.figure(1)
            plt.scatter(X.detach().numpy(),Y.detach().numpy(),color = 'b', s = 1,label = 'trajectory 1')
            plt.figure(2)
            plt.scatter(X_orig.detach().numpy(),Y_orig.detach().numpy(),color = 'b', s = 1,label = 'trajectory 1')
        if m == 1:
            plt.figure(1)
            plt.scatter(X.detach().numpy(),Y.detach().numpy(),color = 'g', s = 1,label = 'trajectory 2')
            plt.figure(2)
            plt.scatter(X_orig.detach().numpy(),Y_orig.detach().numpy(),color = 'g', s = 1,label = 'trajectory 2')
        if m == 2:
            plt.figure(1)
            plt.scatter(X.detach().numpy(),Y.detach().numpy(),color = 'orange', s = 1,label = 'trajectory 3')
            plt.figure(2)
            plt.scatter(X_orig.detach().numpy(),Y_orig.detach().numpy(),color = 'orange', s = 1,label = 'trajectory 3')
            
    t = np.linspace(0,2*np.pi,200)
    plt.figure(1)
    plt.scatter(a[0],a[1])
    plt.scatter(b[0],b[1])
    plt.plot(a[0]+0.1*np.cos(t),a[1]+0.1*np.sin(t))
    plt.plot(b[0]+0.1*np.cos(t),b[1]+0.1*np.sin(t))
    plt.xlim([-1.5,1])
    plt.ylim([-0.5,2])
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.legend()
    plt.rcParams["figure.figsize"] = (5,5)
    plt.savefig('./data/controlled_mueller_T10.pdf')
    
    plt.figure(2)
    plt.scatter(a[0],a[1])
    plt.scatter(b[0],b[1])
    plt.xlim([-1.5,1])
    plt.ylim([-0.5,2])
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.plot(a[0]+0.1*np.cos(t),a[1]+0.1*np.sin(t))
    plt.plot(b[0]+0.1*np.cos(t),b[1]+0.1*np.sin(t))
    plt.legend()
    plt.rcParams["figure.figsize"] = (5,5)
    plt.savefig('./data/uncontrolled_mueller_T10.pdf')
    
    plt.show()
    
    return X,Y,X_orig,Y_orig,I 
        


In [33]:
beta = torch.tensor(1/10)
delt = torch.tensor(1e-5)
X,Y,X_orig,Y_orig,I = plot_traj(3000,3,beta,delt)
